In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation, Resizing, Rescaling
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, GaussianNoise
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


### Variables Globales

In [ ]:
img_width, img_height = 160, 120
train_data_dir = 'Training'
# validation_data_dir = 'Validation'
# test_data_dir = 'Testing'
# nb_validation_samples = 193
validation_data_dir = 'Testing'
# test_data_dir = 'Validation'
nb_validation_samples = 1511
nb_train_samples = 7470
batch_size = 64
epochs = 100

### Explicación: Instanciamos ImageDataGenerator para el conjunto de entrenamiento y prueba.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   vertical_flip=False,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   brightness_range=[0.5, 1.0],
                                   horizontal_flip=True
                                   )

test_datagen = ImageDataGenerator(rescale=1./255)

#### Explicación: Generamos batches de imágenes a partir de los directorios especificados utilizando flow_from_directory.

In [ ]:
training_dataset = tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                            shuffle=True,
                                                            batch_size=batch_size,
                                                            label_mode='categorical',
                                                            image_size=(img_width, img_height),)
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_data_dir,
                                                            shuffle=True,
                                                            label_mode='categorical',
                                                            batch_size=batch_size,
                                                            image_size=(img_width, img_height),)


In [ ]:
validation_batches = tf.data.experimental.cardinality(validation_dataset)
# test_batches = tf.data.experimental.cardinality(test_dataset)
training_batches = tf.data.experimental.cardinality(training_dataset)
print('Number of train batches: %d' % tf.data.experimental.cardinality(training_dataset))
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))


In [ ]:

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')


#### Explicación: Cargamos la arquitectura InceptionV3 desde el módulo de aplicaciones de Keras, con pesos preentrenados en ImageNet, pero sin incluir la capa superior (fully connected).

In [ ]:
base_model = tf.keras.applications.DenseNet201(
    weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False
# Descongelar las últimas 30 capas
# for layer in base_model.layers[-100:]:
#     layer.trainable = True
# base_model.summary()

#### Explicación: Congelamos las capas del modelo base InceptionV3 para que no se actualicen durante el entrenamiento. Transfer Learning. Sólo se actualizarán los pesos de las capas personalizadas. 

#### Explicación: Añadimos capas personalizadas a la salida de la arquitectura base.

In [ ]:
## Probando Actualmente
model = Sequential( [base_model,
                     GlobalAveragePooling2D(),
                        Dense(512, activation='relu'),
                        Dropout(0.5),
                        Dense(7, activation='softmax')
                    ] )


In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
model.summary()

In [ ]:
nb_classes = 7
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation='softmax')(x)

# Create the final model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
model.summary()

In [ ]:
### Original
nb_classes = 7

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation='softmax')(x)
 
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
# model.summary()


In [ ]:
### ChatGPT approach
nb_classes = 7
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)

x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)

x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(nb_classes, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
# model.summary()

In [ ]:
### kenny's approach
nb_classes = 7

inputs = tf.keras.Input(shape=(img_width, img_height, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)

outputs = tf.keras.layers.Dense(nb_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs=outputs)
model.summary()


#### Explicación: Compilamos el modelo con el optimizador Adam, la pérdida de entropía cruzada categórica y la precisión como métrica. 

In [ ]:
nb_classes = 7
adam = Adam(learning_rate=0.1, decay=1e-5)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tfa.metrics.F1Score(num_classes=nb_classes, average='macro')])
# model.summary()


#### Explicación: Creamos un callback de ModelCheckpoint para guardar el modelo con la mejor precisión en el conjunto de validación durante el entrenamiento.

In [ ]:
checkpointer = ModelCheckpoint('model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=15, verbose=1, mode='auto')
red_plateu = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)

# checkpointer = ModelCheckpoint(filepath='model.hdf5', verbose=1, save_best_only=True, monitor='val_loss')
callbacks = [checkpointer, early_stopping, red_plateu]

#### Explicación: Entrenamos el modelo con el método fit_generator utilizando los generadores de imágenes creados antes.

In [ ]:
history = model.fit(train_generator,
          batch_size=batch_size,
        #   steps_per_epoch=nb_train_samples//batch_size,
          epochs=epochs,
          validation_data=validation_generator,
          validation_steps=nb_validation_samples//batch_size,
          callbacks=callbacks)

In [ ]:
nb_validation_samples//batch_size

#### Explicación: Finalmente, guardamos el modelo entrenado.

In [ ]:
model.save('model_gozu.h5')
